In [3]:
# Load the dotenv extension
%load_ext dotenv 
# Load environment variables from a .env file
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# What are we doing?

## Objectives 


* Build a data pipeline that downloads price data from the internet, stores it locally, transforms it into return data, and stores the feature set.
    - Getting the data.
    - Schemas and index in dask.

* Explore the parquet format.
    - Reading and writing parquet files.
    - Read datasets that are stored in distributed files.
    - Discuss dask vs pandas as a small example of big vs small data.
    
* Discuss the use of environment variables for settings.
* Discuss how to use Jupyter notebooks and source code concurrently. 
* Logging and using a standard logger.

## About the Data

+ We will download the prices for a list of stocks.
+ The source is Yahoo Finance and we will use the API provided by the library yfinance.


## Medallion Architecture

+ The architecture that we are thinking about is called Medallion by [DataBricks](https://www.databricks.com/glossary/medallion-architecture). It is an ELT type of thinking, although our data is well-structured.

![Medallion Architecture (DataBicks)](./images/02_medallion_architecture.png)

+ In our case, we would like to optimize the number of times that we download data from the internet. 
+ Ultimately, we will build a pipeline manager class that will help us control the process of obtaining and transforming our data.

![](./images/02_target_pipeline_manager.png)

# Download Data from Yahoo Finance

Yahoo Finance provides information about public stocks in different markets. The library yfinance gives us access to a fair bit of the data in Yahoo Finance. 

These steps are based on the instructions in:

+ [yfinance documentation](https://pypi.org/project/yfinance/)
+ [Tutorial in geeksforgeeks.org](https://www.geeksforgeeks.org/get-financial-data-from-yahoo-finance-with-python/)


+ If required, install: `python -m pip install yfinance`.
+ To download the price history of a stock, first use the following setup:


In [12]:
import pandas as pd
import yfinance as yf
import os
import sys

sys.path.append(os.getenv('SRC_DIR'))

A few things to notice in the code chunk above:

+ Libraries are ordered from high-level to low-level libraries from the package manager (pip in this case, but could be conda, poetry, etc.)
+ The command `sys.path.append("../05_src/)` will add the `../05_src/` directory to the path in the Notebook's kernel. This way, we can use our modules as part of the notebook.
+ Local modules are imported at the end. 
+ The function `get_logger()` is called with `__name__` as recommended by the documentation.

Now, to download the historical price data for a stock, we could use:

In [13]:
px = yf.download('AAPL', start = "2013-12-01", end = "2024-02-01")
px

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2013-12-02,17.175098,19.686787,20.154642,19.672144,19.928572,472544800
2013-12-03,17.645267,20.225714,20.227858,19.917143,19.939285,450968000
2013-12-04,17.604141,20.178572,20.328215,20.029285,20.196428,377809600
2013-12-05,17.694496,20.282143,20.540714,20.228930,20.451786,447580000
2013-12-06,17.448971,20.000713,20.241072,19.984644,20.206785,344352400
...,...,...,...,...,...,...
2024-01-25,193.223404,194.169998,196.270004,193.110001,195.220001,54822100
2024-01-26,191.481934,192.419998,194.759995,191.940002,194.270004,44594000


In [15]:
### play
#type(px)
px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2558 entries, 2013-12-02 to 2024-01-31
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Adj Close, AAPL)  2558 non-null   float64
 1   (Close, AAPL)      2558 non-null   float64
 2   (High, AAPL)       2558 non-null   float64
 3   (Low, AAPL)        2558 non-null   float64
 4   (Open, AAPL)       2558 non-null   float64
 5   (Volume, AAPL)     2558 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 139.9 KB


## Parametrize the download

+ Generally, we will look to separate every parameter and setting from functions.
+ If we had a few stocks, we could cycle through them. We need a place to store the list of tickers (a db or file, for example).
+ Store a csv file with a few stock tickers. The location of the file is a setting, the contents of this file are parameters.
+ Use **environment variables** to pass parameters.

Start by getting a sample of Information Technology stock tickers by applying subindexing and converting the "ticker" column from a pandas object to a list.

In [16]:
# Load all tickers
ticker_file = os.getenv("TICKERS")
tickers = pd.read_csv(ticker_file)


In [20]:
### play
#tickers.info()
tickers.head()

,ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",20883,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",42942,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",20883,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",41274,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",40730,1467373,1989


We can subset our ticker data set using standard indexing techniques. A good reference for this type of data manipulation is Panda's [Documentation](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-and-selecting-data) and [Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook-selection).

In [21]:
idx_tech = tickers['GICS Sector'] == 'Information Technology'
tech_sector = tickers[idx_tech]

In [22]:
### play
idx_tech

0      False
1      False
2      False
3      False
4       True
       ...  
498    False
499     True
500    False
501    False
502    False
Name: GICS Sector, Length: 503, dtype: bool

In [23]:
### play
tech_sector

,ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",40730,1467373,1989
5,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",35555,796343,1982
6,AMD,Advanced Micro Devices,Information Technology,Semiconductors,"Santa Clara, California",42814,2488,1969
12,AKAM,Akamai,Information Technology,Internet Services & Infrastructure,"Cambridge, Massachusetts",39275,1086222,1998
34,APH,Amphenol,Information Technology,Electronic Components,"Wallingford, Connecticut",39721,820313,1932
...,...,...,...,...,...,...,...,...
450,TRMB,Trimble Inc.,Information Technology,Electronic Equipment & Instruments,"Westminster, Colorado",44217,864749,1978
452,TYL,Tyler Technologies,Information Technology,Application Software,"Plano, Texas",44004,860731,1966
467,VRSN,Verisign,Information Technology,Internet Services & Infrastructure,"Dulles, Virginia",38749,1014473,1995
488,WDC,Western Digital,Information Technology,"Technology Hardware, Storage & Peripherals","San Jose, California",39995,106040,1970


From the subset data frame, select one column and convert to list.

In [24]:
tech_tickers = tech_sector['ticker'].to_list()

In [25]:
tech_raw_dt = yf.download(tech_tickers, start = "2000-01-01", end = "2025-01-26")


[*********************100%***********************]  64 of 64 completed


In [26]:
### play
tech_raw_dt

Price        Adj Close                                                  \
Ticker            AAPL         ACN        ADBE         ADI        ADSK   
Date                                                                     
2000-01-03    0.843077         NaN   16.274670   28.095680    8.052908   
2000-01-04    0.771997         NaN   14.909401   26.674351    7.660817   
2000-01-05    0.783294         NaN   15.204174   27.063772    7.178246   
2000-01-06    0.715508         NaN   15.328290   26.323896    6.740915   
2000-01-07    0.749401         NaN   16.072985   27.063772    7.540174   
...                ...         ...         ...         ...         ...   
2025-01-17  229.979996  352.589996  429.989990  219.160004  291.450012   
2025-01-21  222.639999  356.420013  436.359985  221.449997  296.910004   
2025-01-22  223.830002  359.109985  437.320007  224.080002  301.450012   
2025-01-23  223.660004  363.260010  437.279999  228.350006  301.079987   
2025-01-24  222.779999  362.410004  435.380005  217.369995  300.829987   

Price                                                                   ...  \
Ticker            AKAM        AMAT         AMD        ANET        ANSS  ...   
Date                                                                    ...   
2000-01-03  321.250000   23.138010   15.500000         NaN    2.765625  ...   
2000-01-04  300.000000   21.994837   14.625000         NaN    2.687500  ...   
2000-01-05  283.500000   21.171753   15.000000         NaN    2.703125  ...   
2000-01-06  236.125000   21.206036   16.000000         NaN    2.703125  ...   
2000-01-07  248.375000   21.388948   16.250000         NaN    2.703125  ...   
...                ...         ...         ...         ...         ...  ...   
2025-01-17   91.010002  192.050003  121.459999  119.949997  349.929993  ...   
2025-01-21   94.709999  192.869995  122.279999  121.500000  352.070007  ...   
2025-01-22   95.110001  195.509995  123.750000  129.820007  356.630005  ...   
2025-01-23   95.889999  190.699997  123.040001  129.119995  360.359985  ...   
2025-01-24   97.699997  186.750000  122.839996  129.169998  357.369995  ...   

Price        Volume                                                         \
Ticker         SWKS     TDY        TEL      TER     TRMB       TXN     TYL   
Date                                                                         
2000-01-03   512000  315400        NaN  1353500  8025000  10815600  138300   
2000-01-04   292600  444300        NaN  1611800  4963200   7952400  135100   
2000-01-05   411800  109100        NaN  1855800  1930200  12142400  187200   
2000-01-06   385400  120700        NaN   964700  2230800  11758400  107200   
2000-01-07   536000  222100        NaN   809200  2166600  12938800   78900   
...             ...     ...        ...      ...      ...       ...     ...   
2025-01-17  2150600  264400  4102600.0  2253700  1010700   5407500  308000   
2025-01-21  2389300  377400  2568300.0  4083000  1341800   9621700  235400   
2025-01-22  2774100  668700  3162200.0  2262100  1098600   8222000  177300   
2025-01-23  1777300  384700  1417800.0  2214300  1210400  12694700  214000   
2025-01-24  1803300  333500  1017300.0  2254000   895300  15856600  155300   

Price                                  
Ticker         VRSN      WDC     ZBRA  
Date                                   
2000-01-03  2270100  2461900  1055700  
2000-01-04  3002200  7660300   522450  
2000-01-05  6886600  3944600   612225  
2000-01-06  4003200  2468400   263925  
2000-01-07  3803200  9783000   333900  
...             ...      ...      ...  
2025-01-17   503300  7160500   270600  
2025-01-21   781800  7045600   446000  
2025-01-22   674500  9340500   497500  
2025-01-23   630100  7059500   377100  
2025-01-24   563300  6046500   258900  

[6304 rows x 384 columns]

In [27]:
### play
tech_raw_dt.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6304 entries, 2000-01-03 to 2025-01-24
Columns: 384 entries, ('Adj Close', 'AAPL') to ('Volume', 'ZBRA')
dtypes: float64(339), int64(45)
memory usage: 18.5 MB


The data that we downloaded combines several stocks and prices into a single row. We want to parse this arrangement into a dataframe that contains observations about a single stock on a given day per row. To do this, we can use the function [`stack()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html) and re-arrange the indices.

In [28]:
# First, check what tech_raw_dt.stack() looks like.
tech_raw_dt.stack(future_stack=True).reset_index()

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2000-01-03,AAPL,0.843077,0.999442,1.004464,0.907924,0.936384,535796800.0
1,2000-01-03,ACN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-03,ADBE,16.274670,16.390625,16.875000,16.062500,16.812500,7384400.0
3,2000-01-03,ADI,28.095680,45.093750,46.937500,44.000000,46.750000,3655600.0
4,2000-01-03,ADSK,8.052908,8.343750,8.656250,8.031250,8.500000,2845600.0
...,...,...,...,...,...,...,...,...
403451,2025-01-24,TXN,185.520004,185.520004,191.500000,185.029999,190.000000,15856600.0
403452,2025-01-24,TYL,591.929993,591.929993,594.969971,590.210022,591.400024,155300.0
403453,2025-01-24,VRSN,210.729996,210.729996,210.880005,206.009995,206.020004,563300.0
403454,2025-01-24,WDC,67.410004,67.410004,69.440002,67.360001,68.910004,6046500.0


In [29]:
tech_dt = (tech_raw_dt
           .stack(future_stack=True)
           .reset_index()
           .sort_values(['Ticker', 'Date']))
tech_dt.columns.name = None


# Storing Data in CSV



+ We have some data. How do we store it?
+ We can compare two options, CSV and Parqruet, by measuring their performance:

    - Time to save.
    - Space required.

In [30]:
def get_dir_size(path='.'):
    '''Returns the total size of files contained in path.'''
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

In [31]:
import time

In [32]:
temp = os.getenv("TEMP_DATA")
os.makedirs(temp, exist_ok=True)
stock_path = os.path.join(temp, "stock_px.csv")

In [33]:
start = time.time()
tech_dt.to_csv(stock_path, index = False)
end = time.time()

print(f'Writing to dt ({tech_dt.shape})csv took {end - start} seconds.')
print(f'Csv file size { os.path.getsize(stock_path)*1e-6 } MB')

Writing to dt ((403456, 8))csv took 2.4098856449127197 seconds.
Csv file size 41.765691 MB


## Save Data to Parquet

### Dask 

We can work with with large data sets and parquet files. In fact, recent versions of pandas support pyarrow data types and future versions will require a pyarrow backend. The pyarrow library is an interface between Python and the Appache Arrow project. The [parquet data format](https://parquet.apache.org/) and [Arrow](https://arrow.apache.org/docs/python/parquet.html) are projects of the Apache Foundation.

However, Dask is much more than an interface to Arrow: Dask provides parallel and distributed computing on pandas-like dataframes. It is also relatively easy to use, bridging a gap between pandas and Spark. 

In [34]:
import dask.dataframe as dd

c:\Users\zoomf\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [35]:
px_dd = dd.from_pandas(tech_dt, npartitions = len(tech_tickers))
parquet_path = os.path.join(temp, "stock_px.parquet")

start = time.time()
px_dd.to_parquet(parquet_path, engine = "pyarrow")
end = time.time()

print(f'Writing dd ({tech_dt.shape}) to parquet took {end - start} seconds.')
print(f'Parquet file size { get_dir_size(parquet_path)*1e-6 } MB')

Writing dd ((403456, 8)) to parquet took 0.7760813236236572 seconds.
Parquet file size 16.590911 MB


### Parquet files and Dask Dataframes

+ Parquet files are immutable: once written, they cannot be modified.
+ Dask DataFrames are a useful implementation to manipulate data stored in parquets.
+ Parquet and Dask are not the same: parquet is a file format that can be accessed by many applications and programming languages (Python, R, PowerBI, etc.), while Dask is a package in Python to work with large datasets using distributed computation.
+ **Dask is not for everything** (see [Dask DataFrames Best Practices](https://docs.dask.org/en/stable/dataframe-best-practices.html)). 

    - Consider cases suchas small to large joins, where the small dataframe fits in memory, but the large one does not. 
    - If possible, use pandas: reduce, then use pandas.
    - Pandas performance tips apply to Dask.
    - Use the index: it is beneficial to have a well-defined index in Dask DataFrames, as it may speed up searching (filtering) the data. A one-dimensional index is allowed.
    - Avoid (or minimize) full-data shuffling: indexing is an expensive operations. 
    - Some joins are more expensive than others. 

        * Not expensive:

            - Join a Dask DataFrame with a pandas DataFrame.
            - Join a Dask DataFrame with another Dask DataFrame of a single partition.
            - Join Dask DataFrames along their indexes.

        * Expensive:

            - Join Dask DataFrames along columns that are not their index.


# How do we store prices?

+ We can store our data as a single blob. This can be difficult to maintain, especially because parquet files are immutable.
+ Strategy: organize data files by ticker and date. Update only latest month.



In [36]:
# CLean up before start
PRICE_DATA = os.getenv("PRICE_DATA")
import shutil
if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)

In [37]:
for ticker in tech_dt['Ticker'].unique():
    ticker_dt = tech_dt[tech_dt['Ticker'] == ticker]
    ticker_dt = ticker_dt.assign(Year = ticker_dt.Date.dt.year)
    for yr in ticker_dt['Year'].unique():
        yr_dd = dd.from_pandas(ticker_dt[ticker_dt['Year'] == yr],2)
        yr_path = os.path.join(PRICE_DATA, ticker, f"{ticker}_{yr}")
        os.makedirs(os.path.dirname(yr_path), exist_ok=True)
        yr_dd.to_parquet(yr_path, engine = "pyarrow")
    

Why would we want to store data this way?

+ Easier to maintain. We do not update old data, only recent data.
+ We can also access all files as follows.

# Load, Transform and Save 

## Load

+ Parquet files can be read individually or as a collection.
+ `dd.read_parquet()` can take a list (collection) of files as input.
+ Use `glob` to get the collection of files.

In [38]:
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

## Transform

+ This transformation step will create a *Features* data set. In our case, features will be stock returns (we obtained prices).
+ Dask dataframes work like pandas dataframes: in particular, we can perform groupby and apply operations.
+ Notice the use of [an anonymous (lambda) function](https://realpython.com/python-lambda/) in the apply statement.

In [39]:
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

C:\Users\zoomf\AppData\Local\Temp\ipykernel_13952\3086392435.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


In [40]:
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

## Lazy Exection

What does `dd_rets` contain?

In [41]:
dd_rets

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns
npartitions=64,,,,,,,,,,
AAPL,datetime64[ns],float64,float64,float64,float64,float64,float64,int32,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...


+ Dask is a lazy execution framework: commands will not execute until they are required. 
+ To trigger an execution in dask use `.compute()`.

In [42]:
dd_rets.compute()

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns
Ticker,,,,,,,,,,
AAPL,2000-01-03,0.843077,0.999442,1.004464,0.907924,0.936384,535796800.0,2000,NaN,NaN
AAPL,2000-01-04,0.771997,0.915179,0.987723,0.903460,0.966518,512377600.0,2000,0.999442,-0.084310
AAPL,2000-01-05,0.783294,0.928571,0.987165,0.919643,0.926339,778321600.0,2000,0.915179,0.014633
AAPL,2000-01-06,0.715508,0.848214,0.955357,0.848214,0.947545,767972800.0,2000,0.928571,-0.086538
AAPL,2000-01-07,0.749401,0.888393,0.901786,0.852679,0.861607,460734400.0,2000,0.848214,0.047369
...,...,...,...,...,...,...,...,...,...,...
ZBRA,2025-01-17,405.709991,405.709991,407.290009,402.290009,406.040009,270600.0,2025,402.720001,0.007424
ZBRA,2025-01-21,418.070007,418.070007,419.850006,407.619995,407.619995,446000.0,2025,405.709991,0.030465
ZBRA,2025-01-22,420.570007,420.570007,427.760010,419.589996,425.239990,497500.0,2025,418.070007,0.005980


## Save

+ Apply transformations to calculate daily returns
+ Store the enriched data, the silver dataset, in a new directory.
+ Should we keep the same namespace? All columns?

In [43]:
# CLean up before save
FEATURES_DATA = os.getenv("FEATURES_DATA")
if os.path.exists(FEATURES_DATA):
    shutil.rmtree(FEATURES_DATA)
dd_rets.to_parquet(FEATURES_DATA, overwrite = True)

# Optional: from Jupyter to Command Line

+ We have drafted our code in a Jupyter Notebook. 
+ Finalized code should be written in Python modules.

## Object Oriented vs Functional Programming

+ We can use classes to keep parameters and functions together.
+ We *could* use Object Oriented Programming, but parallelization of data manipulation and modelling tasks benefit from *Functional Programming*.
+ An Idea: 

    - [Data Oriented Programming](https://blog.klipse.tech/dop/2022/06/22/principles-of-dop.html).
    - Use the class to bundle together parameters and functions.
    - Use stateless operations and treat all data objects as immutable (we do not modify them, we overwrite them).
    - Take advantage of [`@staticmethod`](https://realpython.com/instance-class-and-static-methods-demystified/).

The code is in `./05_src/`.

Our original design was:

![](./images/02_target_pipeline_manager.png)



The `DataManager` class in `./05_src/data_manager.py` is a simple implementation of the ideas and code discussed in this notebook. The lines below will download data for about 500 stocks from the S&P500. Using this data a few features will be created and stored in the features data set.

First, instantiate an object of class `DataManager`.

In [44]:
from data_manager import DataManager
dm = DataManager()

Download all prices.

In [45]:
dm.download_all()

2025-01-31 18:38:38,153, data_manager.py, 42, INFO, Getting price data for all tickers.
2025-01-31 18:38:38,155, data_manager.py, 51, INFO, Getting tickers from ../../05_src/data/tickers/sp500_wiki.csv
2025-01-31 18:38:38,161, data_manager.py, 57, INFO, Processing all tickers
2025-01-31 18:38:38,163, data_manager.py, 70, INFO, Processing ticker ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 

Finally, add features to the data set and save to a *feature store*.

In [46]:
dm.featurize()

2025-01-31 21:45:13,057, data_manager.py, 114, INFO, Creating features data.
2025-01-31 21:45:13,058, data_manager.py, 124, INFO, Loading price data from ../../05_src/data/prices/
2025-01-31 21:45:17,159, data_manager.py, 133, INFO, Creating features
c:\Users\zoomf\DSI\assignments\production\01_materials\labs\../../05_src\data_manager.py:136: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  features = (price_dd.groupby('Ticker', group_keys=False)
2025-01-31 21:45:17,165, data_manager.py, 158, INFO, Saving features to ../../05_src/data/features/stock_features.parquet
